In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

options = Options()
options.add_argument("--headless")  # Run in the background
options.add_argument("--disable-blink-features=AutomationControlled")  # Avoid bot detection

driver = webdriver.Chrome(service=Service(ChromeDriverManager(driver_version="133.0.6943.99").install()), options=options)
driver.get("https://www.france24.com/ar/%D8%A3%D8%B1%D8%B4%D9%8A%D9%81/2024/")

html = driver.page_source
time.sleep
driver.quit()
# Parse the page source with BeautifulSoup
soup = BeautifulSoup(html, "html.parser")
soup

<html class="webp webp-alpha webp-animation webp-lossless" dir="rtl" lang="ar"><head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0, viewport-fit=cover" name="viewport"/>
<link href="/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/apple-touch-icon.png" rel="apple-touch-icon"/>
<link href="/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
<link href="/favicon-194x194.png" rel="icon" sizes="194x194" type="image/png"/>
<link href="/android-chrome-192x192.png" rel="icon" sizes="192x192" type="image/png"/>
<link href="/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
<link href="/ar/site.manifest" rel="manifest"/>
<link color="#00a7e3" href="/safari-pinned-tab.svg" rel="mask-icon"/>
<title>أرشيف فرانس 24 لسنة 2024: كل المقالات والتسجيلات وفيديوهات الأحداث الدولية</title>
<link href="https://www.france24.com/ar/%D8%A3%D8%B1%D8%B4%D9%8A%D9%81/2024/" rel="canonical"/>
<meta content="https://s.france24.com/media/dis

In [8]:
# Look for containers holding articles, such as divs or sections
links_tag = soup.find_all("a", class_="o-archive-month__days__day__link")  # Replace with actual class name

# Check if it contains any articles
links=[]
dates=[]
for link in links_tag:
    links.append(f"https://www.france24.com{link['href']}")
    dates.append(link.text)
    print(f"https://www.france24.com{link['href']}",link.text)  # Print the HTML of each container

https://www.france24.com/ar/%D8%A3%D8%B1%D8%B4%D9%8A%D9%81/2024/12/31-%D8%AF%D9%8A%D8%B3%D9%85%D8%A8%D8%B1-2024 
                31 ديسمبر 2024
            
https://www.france24.com/ar/%D8%A3%D8%B1%D8%B4%D9%8A%D9%81/2024/12/30-%D8%AF%D9%8A%D8%B3%D9%85%D8%A8%D8%B1-2024 
                30 ديسمبر 2024
            
https://www.france24.com/ar/%D8%A3%D8%B1%D8%B4%D9%8A%D9%81/2024/12/29-%D8%AF%D9%8A%D8%B3%D9%85%D8%A8%D8%B1-2024 
                29 ديسمبر 2024
            
https://www.france24.com/ar/%D8%A3%D8%B1%D8%B4%D9%8A%D9%81/2024/12/28-%D8%AF%D9%8A%D8%B3%D9%85%D8%A8%D8%B1-2024 
                28 ديسمبر 2024
            
https://www.france24.com/ar/%D8%A3%D8%B1%D8%B4%D9%8A%D9%81/2024/12/27-%D8%AF%D9%8A%D8%B3%D9%85%D8%A8%D8%B1-2024 
                27 ديسمبر 2024
            
https://www.france24.com/ar/%D8%A3%D8%B1%D8%B4%D9%8A%D9%81/2024/12/26-%D8%AF%D9%8A%D8%B3%D9%85%D8%A8%D8%B1-2024 
                26 ديسمبر 2024
            
https://www.france24.com/ar/%D8%A3%D8%B1%D8%B4%D9%8A%D9%81

In [9]:
import requests
from bs4 import BeautifulSoup
import csv
def get_data(lien,date):
    try:
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        driver.get(lien)
        html = driver.page_source
        driver.quit()
        # Parse the page source with BeautifulSoup
        soup1 = BeautifulSoup(html, "html.parser")
        try:
            ds=date.split()
            f=""
            for d in ds:
                f=f+"_"+d
            print(f)
            articles = soup1.find_all("li", class_="o-archive-day__list__entry")
            # Open a CSV file to save the data
            with open(f"articles/france24_arabe_articles_archive_{f}.csv", "w", newline="", encoding="utf-8") as file:
                writer = csv.writer(file)
                # Write the header
                writer.writerow(["source",  "title", "content", "description", "url", "published_at", "tags"])
            
                for article in articles:
                    # Source
                    source = "France 24"  # Since it's from France24
                    # Extract the link to the article
                    link_tag = article.find("a",class_="a-archive-link", href=True)
                    article_link = f"https://www.france24.com{link_tag['href']}" if link_tag else "No Link"
                
                    # Extract the article title
                    title_tag = article.find("h2")
                    title = title_tag.text.strip() if title_tag else "No Title"
                    published_at=date
                    
                    content=""
                    description=""
                    tags=""
                    # Write the extracted data to CSV
                    writer.writerow([source, title, content, description, article_link,  published_at, tags])
            
            print("✅ Data saved")
        except (Exception) as e:
                print(f"Error1: {e}")
    except (Exception) as e:
            print(f"Error2: {e}")

In [10]:
import pandas as pd
import time
def complete_data(date):
    ds=date.split()
    f=""
    for d in ds:
        f=f+"_"+d
    print(f)
    df=pd.read_csv(f"articles/france24_arabe_articles_archive_{f}.csv")
    print(df.shape[0])
    for i in range(df.shape[0]):
        if(df.loc[i,'url']):
            try:
                driver = webdriver.Chrome(service=Service(ChromeDriverManager(driver_version="133.0.6943.99").install()), options=options)
                driver.get(df.loc[i,'url'])
                html = driver.page_source
                #time.sleep
                driver.quit()
                # Parse the page source with BeautifulSoup
                soup2 = BeautifulSoup(html, "html.parser")
                soup2
    
                print(i,end='')
                # Extract all news headlines
                contents = soup2.find_all("p", class_="")
                cont = ""
                # Print extracted contents
                for content in contents:
                    cont= cont + "\n"+ content.text.strip()
                df.loc[i,'content']=cont
                # Extract all news headlines
                description = soup2.find("p", class_="t-content__chapo")
                df.loc[i, "description"] = description.text if description else None
                
                times = soup2.find("time", class_="")
                df.loc[i, 'published_at'] = times.text if times else None

                tags=soup2.find_all("div", class_="a-tag-section")
                if tags:
                    df.loc[i,'tags'] = tags[0].text
                else:
                    df.loc[i,'tags'] = None  # Ou "" si tu veux une chaîne vide
            except (Exception) as e:
                print(f"Error: {e}")

    df.to_csv(f"data/france24_arabe_articles_content_archive_{f}.csv")
    print("✅ Data saved")
    


In [110]:
for i in range(len(dates)):
    get_data(links[i],dates[i])

_31_ديسمبر_2024
✅ Data saved
_30_ديسمبر_2024
✅ Data saved
_29_ديسمبر_2024
✅ Data saved
_28_ديسمبر_2024
✅ Data saved
_27_ديسمبر_2024
✅ Data saved
_26_ديسمبر_2024
✅ Data saved
_25_ديسمبر_2024
✅ Data saved
_24_ديسمبر_2024
✅ Data saved
_23_ديسمبر_2024
✅ Data saved
_22_ديسمبر_2024
✅ Data saved
_21_ديسمبر_2024
✅ Data saved
_20_ديسمبر_2024
✅ Data saved
_19_ديسمبر_2024
✅ Data saved
_18_ديسمبر_2024
✅ Data saved
_17_ديسمبر_2024
✅ Data saved
_16_ديسمبر_2024
✅ Data saved
_15_ديسمبر_2024
✅ Data saved
_14_ديسمبر_2024
✅ Data saved
_13_ديسمبر_2024
✅ Data saved
_12_ديسمبر_2024
✅ Data saved
_11_ديسمبر_2024
✅ Data saved
_10_ديسمبر_2024
✅ Data saved
_09_ديسمبر_2024
✅ Data saved
_08_ديسمبر_2024
✅ Data saved
_07_ديسمبر_2024
✅ Data saved
_06_ديسمبر_2024
✅ Data saved
_05_ديسمبر_2024
✅ Data saved
_04_ديسمبر_2024
✅ Data saved
_03_ديسمبر_2024
✅ Data saved
_02_ديسمبر_2024
✅ Data saved
_01_ديسمبر_2024
✅ Data saved
_30_نوفمبر_2024
✅ Data saved
_29_نوفمبر_2024
✅ Data saved
_28_نوفمبر_2024
✅ Data saved
_27_نوفمبر_202

In [11]:
for i in range(216,250):
    print(i)
    complete_data(dates[i])

216
_29_مايو_2024
52
0

C:\Users\HP\AppData\Local\Temp\ipykernel_54636\2221245896.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[i,'content']=cont
C:\Users\HP\AppData\Local\Temp\ipykernel_54636\2221245896.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '
                                                يقبل المواطنون في 27 دولة أوروبية على انتخابات البرلمان الأوروبي، موعد انتخابي هام، ولكن هل تواكب التغطية الإعلامية أهمية الحدث؟ وما مدى مساهمة الإعلام التقليدي والبديل في بلورة الوعي والانخراط السياسي للأوروبيين؟ أسئلة تناقشها رامتان عوايطية مع ضيوف حلقة "منتدى الصحافة" الكاتبة والصحفية شيماء غرسلاوي، والصحفي في جريدة النهارسمير تويني، والصحفي يوسف الهلالي.                    ' has dtype incompatible with float64, please explicitly cast to 

123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051✅ Data saved
217
_28_مايو_2024
33
0

C:\Users\HP\AppData\Local\Temp\ipykernel_54636\2221245896.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '
توفي الفنان اللبناني فؤاد شرف الدين المولود عام 1941 الثلاثاء في أحد مستشفيات بيروت "حيث كان يُعالج عقب العملية التي خضع لها في رأسه"، وفق ما أفادت نقابة ممثلي المسرح والسينما في لبنان في بيان.
ووصفت النقابة الراحل بأنه "النجم الذي عشق لبنان حتى الرمق الأخير"، وأشارت إلى أنه عولج قبل مدة من بعض الآلام في الكلى وتماثل للشفاء وشارك في عملين تلفزيوني وسينمائي، لكنّ "صحته انتكست بعد ذلك وتبيّن أنّه يعاني ورمًا حميدًا في الرأس (...) فخضع لعملية استئصال."
وبعدما تماثل للشفاء، تدهورت حالته فجأة بعد فقدانه الكثير من الدم، فأُدخل المستشفى مجددا "وكان من المقرر أن يخضع غدا (الأربعاء) لعدد من التحاليل والأشعة لكن المنية وافته".
هذا، وذكّر نقيب الممثلين نعمه بدوي عبر صفحته على فيس بوك بأن "النجم الكبير (...) أثرى السينما اللبنانية بالعديد من الأفلام وكذلك الشاشة الصغيرة بالعديد من المسلسلات."
وقال إنه "كان

1234567891011121314151617181920212223242526272829303132✅ Data saved
218
_27_مايو_2024
47
0

...

12345678910111213141516171819202122✅ Data saved
248
_27_أبريل_2024
23
0

C:\Users\HP\AppData\Local\Temp\ipykernel_54636\2221245896.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[i,'content']=cont
C:\Users\HP\AppData\Local\Temp\ipykernel_54636\2221245896.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '
                                                قال وزير المالية الفرنسي برونو لومير يوم الجمعة إن فرنسا مستعدة للمشاركة في تمويل خط كهرباء بقدرة ثلاثة جيجاوات يربط مدينة الدار البيضاء المغربية ببلدة الداخلة في الصحراء الغربية.                    ' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[i, "description"] = description.text if description else None
C:\Users\HP\AppData\Local\Temp\ipykernel_54636\2221245896.py:40: FutureWarning: Sett

12345678910111213141516171819202122✅ Data saved
249
_26_أبريل_2024
47
0

C:\Users\HP\AppData\Local\Temp\ipykernel_54636\2221245896.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '
بدأت الجمعة المرحلة الثانية للتصويت في الهند حيث يحق لنحو مليار شخص التوجه إلى صناديق الاقتراع في انتخابات عامة تضم سبع مراحل بدأت في 19 أبريل/نيسان الماضي وتنتهي في الأول من يونيو/حزيران المقبل. ومن المقرر فرز الأصوات في الرابع من يونيو/حزيران.
ويسعى رئيس الوزراء ناريندرا مودي لولاية ثالثة على التوالي ليعادل بذلك الرقم القياسي بدعم من سجله الاقتصادي وإجراءاته الخاصة بالرعاية الاجتماعية والكبرياء الوطني والقومية الهندوسية وشعبيته.
هذا، وشكل منافسوه تحالفا يضم أكثر من 24 حزبا، ويعدون بالمزيد من الإجراءات المضادة للتمييز والمزيد من المساعدات وإنهاء ما يسمونه بحكم مودي الاستبدادي.
وإلى ذلك، يُنتخب في تصويت اليوم 88 عضوا من أصل 543 عضوا في مجلس النواب، وهناك 160 مليون شخص لهم حق التصويت في هذه المرحلة. وهي تغطي 13 ولاية ومنطقة اتحادية في أكبر دولة في العالم من حيث عدد السكان.
هذا، ويخصص أكثر من نصف

12345678910111213141516171819202122232425262728293031323334353637383940414243444546✅ Data saved


In [17]:
import os
import pandas as pd

# Chemin du dossier contenant les fichiers CSV
dossier_csv = "data"

# Liste pour stocker les DataFrames
dataframes = []

# Lire tous les fichiers CSV du dossier
for fichier in os.listdir(dossier_csv):
    if fichier.endswith(".csv"):
        chemin_fichier = os.path.join(dossier_csv, fichier)
        df = pd.read_csv(chemin_fichier)
        dataframes.append(df)

# Fusionner tous les DataFrames
df_fusionne = pd.concat(dataframes, ignore_index=True)

# Enregistrer le résultat dans un nouveau fichier CSV
df_fusionne.to_csv("fichier_fusionne_2025_2024.csv", index=False)

print("Fusion terminée ! Fichier enregistré sous 'fichier_fusionne.csv'.")


Fusion terminée ! Fichier enregistré sous 'fichier_fusionne.csv'.


In [18]:
df_fusionne.shape

(10756, 10)

In [23]:
df_fusionne.isnull().sum()

Unnamed: 0          0
source              0
title               0
content          6454
description       341
url                 0
published_at       90
tags              634
author          10456
image_url?      10456
dtype: int64

In [62]:
df_fusionne[df_fusionne['description']=='No Description'].shape

(19, 11)

In [ ]:
df_fu

In [64]:
for li in df_fusionne.loc[df_fusionne.description.isnull(), 'url']:
        if(li):
            try:
                driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
                driver.get(li)
                html = driver.page_source
                driver.quit()
                # Parse the page source with BeautifulSoup
                soup2 = BeautifulSoup(html, "html.parser")
                soup2
    
     
                # Extract all news headlines
                contents = soup2.find_all("p", class_="")
                cont = ""
                # Print extracted contents
                for content in contents:
                    cont= cont + "\n"+ content.text.strip()
                df_fusionne[df_fusionne.url==li].content=cont
                # Extract all news headlines
                description = soup2.find("p", class_="t-content__chapo")
                if description:
                    df_fusionne[df_fusionne.url==li].description=description.text
                times = soup2.find("time", class_="")
                df_fusionne[df_fusionne.url==li].published_at=times.text
                tags=soup2.find_all("div", class_="a-tag-section")
                df_fusionne[df_fusionne.url==li].tags=tags[0].text
            except (Exception) as e:
                print(f"Error: {e}")


https://www.france24.com/ar/%D8%A3%D9%85%D8%B1%D9%8A%D9%83%D8%A7/20250102-%D9%87%D9%84-%D9%84%D9%87-%D8%B9%D9%84%D8%A7%D9%82%D8%A9-%D8%A8%D9%87%D8%AC%D9%88%D9%85-%D9%86%D9%8A%D9%88-%D8%A3%D9%88%D8%B1%D9%84%D9%8A%D8%A7%D9%86%D8%B2-%D8%A7%D9%86%D9%81%D8%AC%D8%A7%D8%B1-%D8%B3%D9%8A%D8%A7%D8%B1%D8%A9-%D8%AA%D8%B3%D9%84%D8%A7-%D8%A3%D9%85%D8%A7%D9%85-%D9%81%D9%86%D8%AF%D9%82-%D8%AA%D8%B1%D8%A7%D9%85%D8%A8-%D9%81%D9%8A-%D9%84%D8%A7%D8%B3-%D9%81%D9%8A%D8%BA%D8%A7%D8%B3-%D9%88%D9%85%D9%82%D8%AA%D9%84-%D8%B3%D8%A7%D8%A6%D9%82%D9%87%D8%A7
https://www.france24.com/ar/%D8%A8%D8%B1%D8%A7%D9%85%D8%AC/%D8%A8%D8%A7%D8%A8-%D8%A7%D9%84%D9%88%D9%8A%D8%A8/20250203-%D9%85%D8%B5%D8%B1-%D8%AC%D8%A7%D9%85%D8%B9%D8%A9-%D8%A7%D9%84%D8%A3%D8%B2%D9%87%D8%B1-%D8%AA%D8%B9%D9%84%D9%86-%D8%B9%D9%86-%D8%AA%D8%B9%D8%B1%D9%8A%D8%A8-%D8%A7%D9%84%D8%B7%D8%A8
https://www.france24.com/ar/%D9%81%D9%8A%D8%AF%D9%8A%D9%88/20250204-%D8%A5%D8%B3%D8%B1%D8%A7%D8%A6%D9%8A%D9%84-%D8%AA%D8%B3%D8%AA%D8%B9%D8%AF-%D9%84%D8%A5%D8%B1%D8%B3

In [86]:
df_fusionne[df_fusionne.url=='https://www.france24.com/ar/%D9%81%D9%8A%D8%AF%D9%8A%D9%88/20250205-%D8%A7%D9%84%D8%B3%D9%88%D9%8A%D8%AF-%D9%85%D9%82%D8%AA%D9%84-%D9%86%D8%AD%D9%88-%D8%B9%D8%B4%D8%B1%D8%A9-%D8%A3%D8%B4%D8%AE%D8%A7%D8%B5-%D8%A8%D8%A7%D8%B7%D9%84%D8%A7%D9%82-%D9%86%D8%A7%D8%B1-%D9%81%D9%8A-%D9%85%D8%B1%D9%83%D8%B2-%D8%AA%D8%B9%D9%84%D9%8A%D9%85%D9%8A']

,Unnamed: 0,source,title,content,description,url,published_at,tags
243,30,France 24,السويد في حالة صدمة غداة أسوأ عملية قتل في تار...,NaN,NaN,https://www.france24.com/ar/%D9%81%D9%8A%D8%AF...,\n 05 فبراير 2025\n,\n\n أوروبا...


In [126]:
li='https://www.france24.com/ar/%D8%A3%D8%B1%D8%B4%D9%8A%D9%81/2024/12/21-%D8%AF%D9%8A%D8%B3%D9%85%D8%A8%D8%B1-2024'
if(li):
            try:
                driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
                driver.get(li)
                html = driver.page_source
                driver.quit()
                # Parse the page source with BeautifulSoup
                soup2 = BeautifulSoup(html, "html.parser")
                soup2
    
     
                # Extract all news headlines
                contents = soup2.find_all("p", class_="")
                cont = ""
                # Print extracted contents
                for content in contents:
                    cont= cont + "\n"+ content.text.strip()
                print(contents)
                df_fusionne[df_fusionne.url==li].content=cont
                # Extract all news headlines
                description = soup2.find("p", class_="t-content__chapo")
                print(description.text)
                df_fusionne[df_fusionne.url==li].description=description.text
                print(df_fusionne[df_fusionne.url==li].description)
                times = soup2.find("time", class_="")
                df_fusionne[df_fusionne.url==li].published_at=times.text
                tags=soup2.find_all("div", class_="a-tag-section")
                df_fusionne[df_fusionne.url==li].tags=tags[0].text
            except (Exception) as e:
                print(f"Error: {e}")

Error: Message: session not created: This version of ChromeDriver only supports Chrome version 114
Current browser version is 133.0.6943.99 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0048A813+48355]
	(No symbol) [0x0041C4B1]
	(No symbol) [0x00325358]
	(No symbol) [0x003461AC]
	(No symbol) [0x00341EF3]
	(No symbol) [0x00340579]
	(No symbol) [0x00370C55]
	(No symbol) [0x0037093C]
	(No symbol) [0x0036A536]
	(No symbol) [0x003482DC]
	(No symbol) [0x003493DD]
	GetHandleVerifier [0x006EAABD+2539405]
	GetHandleVerifier [0x0072A78F+2800735]
	GetHandleVerifier [0x0072456C+2775612]
	GetHandleVerifier [0x005151E0+616112]
	(No symbol) [0x00425F8C]
	(No symbol) [0x00422328]
	(No symbol) [0x0042240B]
	(No symbol) [0x00414FF7]
	BaseThreadInitThunk [0x762DFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x76F282AE+286]
	RtlGetAppContainerNamedObjectPath [0x76F2827E+238]



In [82]:
description.text

'\n                                                تعيش\xa0السويد\xa0حالة\xa0من\xa0الصدمة\xa0الأربعاء\xa0غداة\xa0أسوأ\xa0عملية\xa0قتل\xa0جماعي\xa0في\xa0تاريخها\xa0راح\xa0ضحيّتها\xa011\xa0شخصا،\xa0من\xa0بينهم\xa0منفذ\xa0الهجوم،\xa0ووقعت\xa0في\xa0مركز\xa0تعليمي\xa0للبالغين\xa0في\xa0أوربرو\xa0(وسط)\xa0مثيرة\xa0العديد\xa0من\xa0الأسئلة\xa0من\xa0دون\xa0إجابات.                    '

In [130]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

options = webdriver.ChromeOptions()
options.add_argument("--headless")

try:
    # Manually set the Chrome version
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager(driver_version="133.0.6943.99").install()), options=options
    )

    li = "https://www.france24.com/ar/%D8%A3%D8%B1%D8%B4%D9%8A%D9%81/2024/12/21-%D8%AF%D9%8A%D8%B3%D9%85%D8%A8%D8%B1-2024"
    driver.get(li)
    html = driver.page_source
    driver.quit()

    soup2 = BeautifulSoup(html, "html.parser")

    contents = soup2.find_all("p", class_="")
    cont = "\n".join(content.text.strip() for content in contents)

    print("Extracted Contents:", cont)

except Exception as e:
    print(f"Error: {e}")


Extracted Contents: 


In [132]:
description = soup2.find("p", class_="t-content__chapo")
print(description.text)

AttributeError: 'NoneType' object has no attribute 'text'